In [1]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import time
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [2]:
province = ['กระบี่', 'กรุงเทพมหานคร', 'กาญจนบุรี', 'กาฬสินธุ์', 'กำแพงเพชร', 'ขอนแก่น', 'จันทบุรี', 'ฉะเชิงเทรา', 'ชลบุรี', 'ชัยนาท', 'ชัยภูมิ',
            'ชุมพร', 'เชียงราย', 'เชียงใหม่', 'ตรัง', 'ตราด', 'ตาก', 'นครนายก', 'นครปฐม', 'นครพนม', 'นครราชสีมา', 'นครศรีธรรมราช', 'นครสวรรค์', 'นนทบุรี', 'นราธิวาส', 'น่าน', 'บึงกาฬ',
            'บุรีรัมย์', 'ปทุมธานี', 'ประจวบคีรีขันธ์', 'ปราจีนบุรี', 'ปัตตานี', 'พระนครศรีอยุธยา', 'พะเยา', 'พังงา', 'พัทลุง', 'พิจิตร', 'พิษณุโลก', 'เพชรบุรี', 'เพชรบูรณ์', 'แพร่', 'ภูเก็ต',
            'มหาสารคาม', 'มุกดาหาร', 'แม่ฮ่องสอน', 'ยโสธร', 'ยะลา', 'ร้อยเอ็ด', 'ระนอง', 'ระยอง', 'ราชบุรี', 'ลพบุรี', 'ลำปาง', 'ลำพูน', 'เลย', 'ศรีสะเกษ', 'สกลนคร', 'สงขลา', 'สตูล',
            'สมุทรปราการ', 'สมุทรสงคราม', 'สมุทรสาคร', 'สระแก้ว', 'สระบุรี', 'สิงห์บุรี', 'สุโขทัย', 'สุพรรณบุรี', 'สุราษฎร์ธานี', 'สุรินทร์', 'หนองคาย', 'หนองบัวลำภู', 'อ่างทอง',
            'อำนาจเจริญ', 'อุดรธานี', 'อุตรดิตถ์', 'อุทัยธานี', 'อุบลราชธานี']

In [3]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('../code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [4]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
        
    except :
        postcode = ""
    
    time.sleep(1)
    
    return province,district,subdistrict,postcode

In [5]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [6]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [7]:
word_spilt_general = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','ตำบล','อำเภอ','จังหวัด']
word_spilt_specialcase = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','ตำบล','อำเภอ','จังหวัด']
#word_spilt_bangkok = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','จังหวัด']
remove_flag_list = []

def splitAddress(address,lat,lng):
    #print(address)
    # province id 1 = bangkok
    
    #searching for replace wrong keyword
    str_address = address.replace("  "," ")
    #print(str_address)

    #remove quote 
    if(re.findall(r'["]',str_address)) :
        str_address = re.sub(r'["]',"",str_address)
    
    #print(str_address)
    
    
    # re.findall(r'\w+[.]\w+(?=@)', s) 
    
    if(re.findall(r'\ต+[.]',str_address) or re.findall(r'\อ+[.]',str_address) or re.findall(r'\จ+[.]',str_address) ) :
        #print("wrong keyword!")
        flag_tum = False
        flag_aumper = False
        flag_province = False
        if(re.findall(r"\ต.",str_address)) :
            flag_tum  = True

        if(re.findall(r"\อ.",str_address)) :
            flag_aumper = True

        if(re.findall(r"\จ.",str_address)) :
            flag_province = True
        
        #print(flag_tum)
        #print(flag_aumper)
        #print(flag_province)
        str_address = str_address.split(" ")
        
        if flag_tum :
            str_address[-4] = str_address[-4].replace("ต.", "ตำบล")
            #print(str_address[-4])
        if flag_aumper :
            str_address[-3] = str_address[-3].replace("อ.", "อำเภอ")
            #print(str_address[-3])
        if flag_province :
            str_address[-2] = str_address[-2].replace("จ.", "จังหวัด")
            #print(str_address[-2])
        
        temp = ""
        for i in str_address :
            temp = temp + i + " "
        str_address = temp
        
    #print(str_address)
    
    if not (re.findall(r"\จังหวัด",str_address)) :
        #print("test")
        str_address = str_address.split(" ")
        #print(str_address)
        str_address[-2] = "จังหวัด"+str_address[-2]
        temp = ""
        for i in str_address :
            temp = temp + i + " "
        str_address = temp

    str_address = str_address.replace(' ','')
   
    for i in range (len(word_spilt_general)):
        #print(word_spilt_general[i])
        if re.search(word_spilt_general[i],str_address) :
            #print("found!")
            str_address = re.sub(word_spilt_general[i]," ",str_address)
            str_address = str_address.lstrip()
            remove_flag_list.append(True)
        else :
            
            #print("not found!")
            remove_flag_list.append(False)
    
    str_address = str_address.rstrip() 

    #print("เลขที่, [0-9] , / , - , ถนน , หมู่ที่ , หมู่ , แขวง , เขต , ตำบล , อำเภอ , จังหวัด ")
    #print(remove_flag_list)

    str_address = str_address.replace("  "," ")
    str_address = str_address.split(" ")

    #print(str_address)
    subdistrict = str_address[-3]
    district = str_address[-2]
    province = str_address[-1]
    
    is_find = False
    
    if(province == "กรุงเทพฯ"):
        province = province.replace("กรุงเทพฯ","กรุงเทพมหานคร")
    if(province not in list_province_in_thai):
        print(str_address)
        is_find = True
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        postcode = extracteachDetail(postcode)
        province = extracteachDetail(province)
        district = extracteachDetail(district)
        subdistrict = extracteachDetail(subdistrict)
    if(district not in list_district_in_thai):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    if(subdistrict not in subdistrict):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
            
        
    
    #print("แขวง,ตำบล: "+ str_address[-3])
    #print("เขต,อำเภอ: "+ str_address[-2])
    
    remove_flag_list.clear()
    
    return subdistrict,district,province
    

In [8]:
def getStoretypename(id):
    switcher = {
        "1": "Big C Supercenter",
        "3": "Big C Market",
        "4": "Big C Foodplace",
        "5": "Big C Mini",
        "6": "Pure Pharmacy",
        "7": "Depot"
    }
    return switcher.get(id)


In [9]:
def getStoretype(id):
    switcher = {
        "1": "Hypermarket",
        "3": "Supermarket",
        "4": "Speciality Store",
        "5": "Convenience Stores",
        "6": "Speciality Store",
        "7": "Speciality Store"
    }
    return switcher.get(id)


In [10]:
# detail BigC data
'''
- Url  "https://corporate.bigc.co.th/include/get_store.php"
- return to List type that contain all data
- each list contain dict that contain each value of store
- keys ['id_store_location', 'store_no', 'name_th', 'name_en', 'address_th', 'address_en', 'tel', 'mobile', 'line_url', 'PROVINCE_ID', 'business_hour_th',
     'business_hour_en', 'latitude', 'longitude', 'id_shop_type', 'publish_date', 'create_date', 'update_date', 'active', 'name']
- example json_data[0]['name_th']
- output
    BigC Extra ลาดพร้าว
'''

'\n- Url  "https://corporate.bigc.co.th/include/get_store.php"\n- return to List type that contain all data\n- each list contain dict that contain each value of store\n- keys [\'id_store_location\', \'store_no\', \'name_th\', \'name_en\', \'address_th\', \'address_en\', \'tel\', \'mobile\', \'line_url\', \'PROVINCE_ID\', \'business_hour_th\',\n     \'business_hour_en\', \'latitude\', \'longitude\', \'id_shop_type\', \'publish_date\', \'create_date\', \'update_date\', \'active\', \'name\']\n- example json_data[0][\'name_th\']\n- output\n    BigC Extra ลาดพร้าว\n'

In [11]:
def cleanDataBigC():
    url = "https://corporate.bigc.co.th/include/get_store.php"
    headers = {'Accept': 'application/json, text/plain, */*'}
    #form_data = {"type":"ร้านขายยาเพรียว"}
    #response = requests.post(url, headers=headers , data=form_data)
    response = requests.post(url, headers=headers)
    payload = response.json()
    #i = 206
    i = 0
    #print("length: " + str(len(payload)))
    while i < len(payload):
        #print("#####################################")
        #print("item at: " + str(i))
        store_id = payload[i]['store_no']
        name = payload[i]['name_th']
        address = payload[i]['address_th']
        str_address = str(address)
        lat = payload[i]['latitude']
        lng = payload[i]['longitude']
        if(name == "MBC ประชาอุทิศ 72, กรุงเทพมหานคร"):
            lat = '13.629144934390737'
            lng = '100.4983715397938'
        subdistrict,district,province  = splitAddress(str_address,lat,lng)
        store_type_id = payload[i]['id_shop_type']
        store_type_name = getStoretypename(store_type_id)
        store_type = getStoretype(store_type_id)
    
        dict_payload = { "name": name,
                        "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
        
        if(store_type_id == "1"):
            list_BigCSupercenter.append(dict_payload)
        elif(store_type_id == "3"):
            list_BigCMarket.append(dict_payload)
        elif(store_type_id == "4"):
            list_BigCFoodplace.append(dict_payload)
        elif(store_type_id == "5"):
            list_BigCMini.append(dict_payload)
        elif(store_type_id == "6"):
            list_PurePharmacy.append(dict_payload)
        elif(store_type_id == "7"):
            list_Depot.append(dict_payload)
        
        i = i + 1
    

In [12]:
list_BigCMarket = []
list_BigCSupercenter = []
list_BigCFoodplace = []
list_BigCMini = []
list_PurePharmacy = []
list_Depot = []
cleanDataBigC()

['ปากน้ำ', 'บางคล้า', 'ฉะเชิงเทรา', '', '', '+H']
['ในเมือง', 'เมือง', 'ขอนแก่น']
['เพชรเกษม', 'ห้วยจรเข้', 'เมือง', 'นครปฐม']
['สวรรค์วิถี', 'ปากน้ำโพ', 'เมือง', 'นครสวรรค์']
[',', '', '', '', ',', 'เฉลิมพระเกียรติร.', 'ประเวศ,', 'กรุงเทพมหานคร']
['เลี่ยงเมือง', 'บางกุ้ง', 'เมือง', 'สุราษฎร์ธานี']
['หลักเมือง', 'ในเมือง', 'เมือง', 'สุรินทร์']
['หมากแข้ง', 'เมือง', 'อุดรธานี']
['ถ.เคหะร่มเกล้า', 'คลองสองต้นนุ่น', 'ลาดกระบัง', 'กรุงเทพ']
['รอบเวียง', 'เมือง', 'เชียงราย']
['ซอยประชาอุทิศ', '', 'ทุ่งคร', 'ทุ่งครุ', 'กรุงเทพมาหานคร']
['ระพีพัฒน์ฝั่งซ้าย', 'หนองแค', 'หนองแค\n', 'สระบุรี']
['บ่อผุด', 'เกาะสมุย', 'สุราษฎ์ธานี']
['ซอย', 'บ้านทางรถไฟฝั่งตะวันตก', 'หัวหิน', 'หัวหิน\n', 'ประจวบคีรีขันธ์']
['บางกุ้ง', 'เมืองสุราษฎร์ธานี\n', 'สุราษฎร์ธานี']
['แสนสุข', 'วารินชำราบ\n', 'อุบลราชธานี']
['ประตูลอด', 'ในเมือง', 'เมืองนครศรีธรรมรา', 'นครศรีธรรมราช']
['พ่อขุนทะเล', 'มะขามเตี้ย', 'เมืองสุราษฎร์ธานี\n', 'สุราษฎร์ธานี']
['ราชการดำเนิน', 'ในเมือง', 'เมืองร้อยเอ็ด\n', 'ร้อยเอ็ด']
['งามวงศ์วาน',

In [13]:
#list_BigCSupercenter  "1": "Big C Supercenter",
#        "3": "Big C Market",
#        "4": "Big C Foodplace",
#        "5": "Big C Mini",
#        "6": "Pure Pharmacy",
#        "7": "Depot"

In [14]:
now = datetime.today()
with open(f"../trash/BigCMarket{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_BigCMarket, outfile)
with open(f"../trash/BigCSupercenter{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_BigCSupercenter, outfile)
with open(f"../trash/BigCFoodplace{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_BigCFoodplace, outfile)
with open(f"../trash/BigCMini{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_BigCMini, outfile)
with open(f"../trash/PurePharmacy{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_PurePharmacy, outfile)
with open(f"../trash/Depot{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_Depot, outfile)

In [15]:
df_json = pd.read_json(f"../trash/BigCMarket{str(now.date())}.json")
df_json.to_csv(f"../csv/BigC/BigCmarket/BigCMarket{str(now.date())}.csv")
df_json.to_excel(f"../excel/BigC/BigCmarket/BigCMarket{str(now.date())}.xlsx")

df_json = pd.read_json(f"../trash/BigCSupercenter{str(now.date())}.json")
df_json.to_csv(f"../csv/BigC/BigCSupercenter/BigCSupercenter{str(now.date())}.csv")
df_json.to_excel(f"../excel/BigC/BigCSupercenter/BigCSupercenter{str(now.date())}.xlsx")

df_json = pd.read_json(f"../trash/BigCFoodplace{str(now.date())}.json")
df_json.to_csv(f"../csv/BigC/BigCFoodplace/BigCFoodplace{str(now.date())}.csv")
df_json.to_excel(f"../excel/BigC/BigCFoodplace/BigCFoodplace{str(now.date())}.xlsx")

df_json = pd.read_json(f"../trash/BigCMini{str(now.date())}.json")
df_json.to_csv(f"../csv/BigC/BigCMini/BigCMini{str(now.date())}.csv")
df_json.to_excel(f"../excel/BigC/BigCMini/BigCMini{str(now.date())}.xlsx")

df_json = pd.read_json(f"../trash/PurePharmacy{str(now.date())}.json")
df_json.to_csv(f"../csv/BigC/PurePharmacy/PurePharmacy{str(now.date())}.csv")
df_json.to_csv(f"../excel/BigC/PurePharmacy/PurePharmacy{str(now.date())}.xlsx")
#df_json = pd.read_json(f"../dataCleandone/BigC/Depot{str(now.date())}.json")
#df_json.to_csv(f"../dataCleandone/done/Depot{str(now.date())}.xlsx")

In [17]:
'''#toDBBigC
bf = pd.read_csv(f'../csv/BigC/BigCFoodplace/BigCFoodplace{str(now.date())}.csv', header = 0)
bm = pd.read_csv(f'../csv/BigC/BigCMarket/BigCMarket{str(now.date())}.csv', header = 0)
bmn = pd.read_csv(f'../csv/BigC/BigCMini/BigCMini{str(now.date())}.csv', header = 0)
bs = pd.read_csv(f'../csv/BigC/BigCSupercenter/BigCSupercenter{str(now.date())}.csv', header = 0)
pp = pd.read_csv(f'../csv/BigC/PurePharmacy/PurePharmacy{str(now.date())}.csv', header = 0)
#print(bf)
engine = create_engine('mysql://root:@localhost:3306/bigcfoodplace')
with engine.connect() as conn, conn.begin():
    bf.to_sql(f'BigCFoodplace{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/bigcmarket')
with engine.connect() as conn, conn.begin():
    bm.to_sql(f'BigCMarket{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/minibigc')
with engine.connect() as conn, conn.begin():
    bmn.to_sql(f'BigCMini{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/bigcsupercenter')
with engine.connect() as conn, conn.begin():
    bs.to_sql(f'BigCSupercenter{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/purepharmacy')
with engine.connect() as conn, conn.begin():
    pp.to_sql(f'PurePharmacy{str(now.date())}', conn, if_exists='append', index=False)
conn.close()'''

"#toDBBigC\nbf = pd.read_csv(f'../csv/BigC/BigCFoodplace/BigCFoodplace{str(now.date())}.csv', header = 0)\nbm = pd.read_csv(f'../csv/BigC/BigCMarket/BigCMarket{str(now.date())}.csv', header = 0)\nbmn = pd.read_csv(f'../csv/BigC/BigCMini/BigCMini{str(now.date())}.csv', header = 0)\nbs = pd.read_csv(f'../csv/BigC/BigCSupercenter/BigCSupercenter{str(now.date())}.csv', header = 0)\npp = pd.read_csv(f'../csv/BigC/PurePharmacy/PurePharmacy{str(now.date())}.csv', header = 0)\n#print(bf)\nengine = create_engine('mysql://root:@localhost:3306/bigcfoodplace')\nwith engine.connect() as conn, conn.begin():\n    bf.to_sql(f'BigCFoodplace{str(now.date())}', conn, if_exists='append', index=False)\nconn.close()\nengine = create_engine('mysql://root:@localhost:3306/bigcmarket')\nwith engine.connect() as conn, conn.begin():\n    bm.to_sql(f'BigCMarket{str(now.date())}', conn, if_exists='append', index=False)\nconn.close()\nengine = create_engine('mysql://root:@localhost:3306/minibigc')\nwith engine.conne